In [1]:
!pip install transformers 

     |████████████████████████████████| 1.9MB 3.5MB/s 
     |████████████████████████████████| 3.2MB 18.6MB/s 
     |████████████████████████████████| 890kB 34.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=001e472d1bde70e4ad794e849496fd54814a8e0acdd7967df42a0af62576e934
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [50]:
!cp -r drive/MyDrive/bugurt_model/ .

cp: -r not specified; omitting directory 'drive/MyDrive/bugurt_model/'


In [3]:
!unzip data.zip

Archive:  data.zip
   creating: bugurt_model/
  inflating: bugurt_model/config.json  
  inflating: bugurt_model/pytorch_model.bin  
  inflating: bugurt_model/tokenizer_config.json  
  inflating: bugurt_model/special_tokens_map.json  
  inflating: bugurt_model/added_tokens.json  
  inflating: bugurt_model/vocab.json  
  inflating: bugurt_model/merges.txt  
  inflating: bugurt_model/training_args.bin  
 extracting: bugurt_model/eval_results_clm.txt  


In [23]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')
model = GPT2LMHeadModel.from_pretrained('bugurt_model')
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
model.to("cuda")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [29]:
import copy

def prettify_text(text):
  pretty_text  = text.replace(" @ ","\n@\n")
  return pretty_text

bad_word_ids = [
    [203], # \n
    [225], # weird space 1
    [28664], # weird space 2
    [13298], # weird space 3
    [206], # \r
    [49120], # html
    [25872], # http
    [3886], # amp
    [38512], # nbsp
    [10], # &
    [5436], # & (another)
    [5861], # http
    [372], # yet another line break
    [421, 4395], # МСК
    [64], # \
    [33077], # https
    [1572], # ru
    [11101], # Источник
]

def gen_fragment(context, bad_word_ids=bad_word_ids, print_debug_output=False):
    input_ids = tokenizer.encode(context, add_special_tokens=False, return_tensors="pt").to("cuda")
    input_ids = input_ids[:,:]
    input_size = input_ids.size(1)
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=500 + input_size,
        min_length=100 + input_size,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1,
        temperature=0.9,
        bad_words_ids=bad_word_ids,
    )
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()
    generated_sequence = output_sequences[0].tolist()[input_size:]
    if print_debug_output:
        for idx in generated_sequence:
            print(idx, tokenizer.decode([idx], clean_up_tokenization_spaces=True).strip())
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    text = text[: text.find("<EOS>")]
    return context + text

def gen_bugurt(context, bad_word_ids, print_debug_output=False):
    bad_word_ids_bugurt = copy.copy(bad_word_ids)
    return prettify_text(gen_fragment(context, bad_word_ids_bugurt, print_debug_output=False))




In [48]:
bugurts = []
from tqdm import trange
for i in trange(5):
  current_text = "<BOS>"
  current_text = gen_bugurt(current_text, bad_word_ids)
  bugurts.append(current_text)



  0%|          | 0/5 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 20%|██        | 1/5 [00:10<00:43, 11.00s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 40%|████      | 2/5 [00:21<00:32, 10.99s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 60%|██████    | 3/5 [00:32<00:22, 11.00s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 80%|████████  | 4/5 [00:43<00:10, 10.98s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100%|██████████| 5/5 [00:54<00:00, 10.96s/it]


In [53]:
print(bugurts[3])

<BOS> УМЕР В 14 ЛЕТ
@
ТЕПЕРЬ ТЫ ГОРДО ЗВАННЫЙ ЧЕЛОВЕК, ЖИВЕШЬ В ВЕЛИКОЙ ОТЕЧЕСТВЕННОЙ И ПРОШЕДШИЙ ВОЙНУ С 1941 ПО 1945 ГОД, ВЫЖИВШИЙ В ТВОЕМ ГОРОДЕ БЫЛ АДМИНИСТРАТОРОМ ОТЕЧЕСТВЕННОЙ ШКОЛЫ
@
@ В ТВОЕМ ГОРОДЕ ЗАХВАТИЛИ МИНИСТЕРСТВО И ТЕБЯ САЖАЮТ ЗА ХУЛИГАНСТВО
@
ТВОЙ ПОДРУГ, КОТОРЫЙ ТЕБЯ ПРОВОДИЛ, ПОЛУЧАЕТ ВЫЗОВ В СУД
@
ПОСЛЕ ВЫЗОВА, ПОКА ТЫ БЫЛ У МАМКИ С ПАПОЙ, ТЕБЯ ПЕРЕЕЗЖАЮТ В ДРУГОЙ ГОРОД
@
ТЫ ГОТОВ ВСТАТЬ С КРОВАТИ И ПРИНЯТЬ УЧАСТИЕ В УЧЕБЕ И ЗАБОТЕ О ДЕТЯХ, НО ТЕБЯ ВЫПИЛИВАЮТ ИЗ ДОМА
@
ТЫ ЕЩЕ И ОДИНОК, СКОРО ВСЕ ТВОИ ДРУЗЬЯ И ДАЖЕ НЕБО ЗАВИДУЮТ ТЕБЕ, А ТЫ И НЕ ПРОТИВ
@
ТЫ УЖЕ ЗНАЕШЬ, ЧТО ДЕЛАТЬ
@
ТЫ ХОДИШЬ В ДЕТСКИЙ ДОМ, РАЗВИВАЕШЬ ТВОИХ ДРУЗЕЙ
@
ТЫ НАБИРАЕШЬСЯ МУЖЕСТВА И ПОЛУЧАЕШЬ ПОВЕСТКУ В ВОЕНКОМАТ
@
ТЫ ДОЛЖЕН ОТПРАВИТЬСЯ ВОЕВАТЬ, НО ТЕБЯ ПЕРЕВОДЯТ В АДМИНИСТРАТИВНЫЙ КОЛОННУ, ГДЕ ТЫ УЖЕ ГОТОВ УНИЧТОЖАТЬ ВСЮ НЕМЕЦКУЮ АРМИЮ, ГДЕ ТЫ ГОТОВ УБИТЬ ДАЖЕ НЕМЦЕВ, НО У ТЕБЯ ЗАКАНЧИВАЮТСЯ ДЕНЬГИ НА ПОЕЗДКУ В США
@
ТЫ ПОЕХАЛ
@
ТЫ УЖЕ ПРЕДВКУШАЕШЬ, КАК БУД
